In [1]:
from typing import Callable, Optional, List, Tuple

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from statsmodels.distributions.empirical_distribution import ECDF
from tqdm import tqdm

In [2]:
def do_radical_search(
        data: pd.DataFrame,
        design: pd.DataFrame,
        threshold: float,
        control: str,
        control_based: bool
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Identify the samples with extreme feature values either based on the entire dataset or control population.

    :param data: Dataframe containing the gene expression values
    :param design: Dataframe containing the design table for the data
    :param threshold: Threshold for choosing patients that are "extreme" w.r.t. the controls
    :param control: label used for representing the control in the design table of the data
    :param control_based: The scoring is based on the control population instead of entire dataset
    :return: Dataframe containing the Single Sample scores using radical searching
    """
    # Transpose matrix to get the patients as the rows
    data_transpose = data.transpose()

    # Give each label an integer to represent the labels during classification
    label_mapping = {
        key: val
        for val, key in enumerate(np.unique(design['Target']))
    }

    # Make sure the number of rows of transposed data and design are equal
    assert len(data_transpose) == len(design), 'Data doesnt match the design matrix'

    # Create a dataframe initialized with 0's [patients x features]
    output_df = pd.DataFrame(0, index=data_transpose.index, columns=data_transpose.columns)

    # Values that are greater than the threshold or lesser than negative threshold are considered as extremes.
    upper_thresh = 1 - (threshold / 100)
    lower_thresh = (threshold / 100)

    if control_based:
        controls = data_transpose[list(design.Target == control)]

        # Calculate the empirical cdf for every gene and get the cdf score for the data
        control_ecdf = controls.apply(_get_ecdf, step=False, extrapolate=True).values
        cdf_score = _apply_func(data_transpose, control_ecdf).fillna(0)

        # Check if each patient's feature is over or under expressed compared to the control population
        output_df = pd.DataFrame(np.where(cdf_score.values > upper_thresh, 1, output_df.values))
        output_df = pd.DataFrame(np.where(cdf_score.values < lower_thresh, -1, output_df.values))

    else:
        # Calculate the empirical cdf for every gene and get the cdf score for the data
        feature_to_ecdf = {
            feature: _get_ecdf(data_transpose[feature])
            for feature in data_transpose
            if len(data_transpose[feature].unique()) > 1  # Check not all values are the same
        }

        # Iterate over patients and check if any of its features is significant
        for patient_index, features in data_transpose.iterrows():

            # Iterate over patient features
            for feature, value in features.items():

                # Skip if feature has no calculated eCDF
                if feature not in feature_to_ecdf:
                    continue

                # Calculate position of the patient in the distribution of the feature
                patient_position_in_distribution = float(feature_to_ecdf[feature]([value])[0])

                if patient_position_in_distribution <= lower_thresh:
                    output_df[feature][patient_index] = -1

                if patient_position_in_distribution > upper_thresh:
                    output_df[feature][patient_index] = 1

    output_df.columns = data.index
    output_df.index = data.columns

    summary_df = output_df.apply(pd.Series.value_counts)

    # Add labels to the data samples
    label = design['Target'].map(label_mapping)
    label.reset_index(drop=True, inplace=True)

    output_df['label'] = label.values

    return output_df, summary_df

In [3]:
def _get_ecdf(
        obs: np.array,
        side: Optional[str] = 'right',
        step: Optional[bool] = True,
        extrapolate: Optional[bool] = False
) -> Callable:
    """Calculate the Empirical CDF of an array and return it as a function.

    :param obs: Observations
    :param side: Defines the shape of the intervals constituting the steps. 'right' correspond to [a, b) intervals
        and 'left' to (a, b]
    :param step: Boolean value to indicate if the returned value must be a step function or an continuous based on
        interpolation or extrapolation function
    :param extrapolate: Boolean value to indicate if the continuous must be based on extrapolation
    :return: Empirical CDF as a function
    """
    if step:
        return ECDF(x=obs, side=side)
    else:
        obs = np.array(obs, copy=True)
        obs.sort()

        num_of_obs = len(obs)

        y = np.linspace(1. / num_of_obs, 1, num_of_obs)

        if extrapolate:
            return interp1d(obs, y, bounds_error=False, fill_value="extrapolate")
        else:
            return interp1d(obs, y)

In [4]:
def _apply_func(
        df: pd.DataFrame,
        func_list: List[Callable]
) -> pd.DataFrame:
    """Apply functions from the list (in order) on the respective column.

    :param df: Data on which the functions need to be applied
    :param func_list: List of functions to be applied
    :return: Dataframe which has been processed
    """
    final_df = pd.DataFrame()

    new_columns = [index for index, _ in enumerate(df.columns)]
    old_columns = list(df.columns)

    df.columns = new_columns

    for idx, i in enumerate(tqdm(df.columns, desc='Searching for radicals: ')):
        final_df[i] = np.apply_along_axis(func_list[idx], 0, df[i].values)

    final_df.columns = old_columns

    return final_df

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
%cd drive/MyDrive/capstone
%pwd

/content/drive/MyDrive/capstone


'/content/drive/MyDrive/capstone'

In [7]:
data = pd.read_csv('./output/metadata_2.csv',sep='\t')
data.set_index('GENE_SYMBOL',inplace=True)
design = pd.read_csv('./output/design_2.csv',sep='\t')
design.rename(columns={'visit':'Target'},inplace=True)
data.head()

,GSM6249277,GSM6249278,GSM6249280,GSM6249281,GSM6249282,GSM6249284,GSM6249285,GSM6249286,GSM6249289,GSM6249290,...,GSM6249835,GSM6249836,GSM6249837,GSM6249838,GSM6249839,GSM6249840,GSM6249841,GSM6249842,GSM6249843,GSM6249844
GENE_SYMBOL,,,,,,,,,,,,,,,,,,,,,
A1BG,4.2906,4.1154,4.0980,5.0073,3.6093,4.1227,4.4585,4.8447,4.2919,3.7814,...,3.1007,3.2824,3.2758,3.2983,3.5604,3.3141,3.4142,3.5609,3.4108,3.4986
A1CF,7.1308,3.6630,7.2920,6.5715,7.2546,7.7518,5.6207,6.2695,7.3611,8.2484,...,8.7742,9.0978,8.7833,9.2721,9.1731,8.8476,8.2624,9.2099,8.6293,8.8979
A2BP1,4.6496,4.7958,5.2102,5.5799,5.1242,5.1014,4.9252,5.3261,5.1597,5.0481,...,4.7368,4.7033,4.7554,5.1079,4.7436,4.4994,4.5251,4.7414,5.0985,4.6558
A2LD1,3.9874,3.6829,4.0572,4.5319,3.8283,3.7058,3.5801,4.1014,3.7704,4.0662,...,3.5877,4.2188,3.6526,3.9762,3.7061,3.8280,4.5468,3.4888,4.3177,3.5686
A2M,10.8287,11.1326,10.4526,10.2151,10.6220,10.3761,11.1019,10.8756,11.1937,10.2104,...,9.9136,9.3516,9.2753,8.6767,8.8701,9.4490,9.7396,8.8704,9.5386,9.6500


In [15]:
design

,geo_accession,Target
0,GSM6249277,baseline
1,GSM6249278,baseline
2,GSM6249280,baseline
3,GSM6249281,baseline
4,GSM6249282,baseline
...,...,...
377,GSM6249840,control
378,GSM6249841,control
379,GSM6249842,control
380,GSM6249843,control


In [9]:
data.shape

(20935, 382)

In [17]:
output_df, summary_df = do_radical_search(
    data = data,
    design = design,
    threshold=5,
    control="control",
    control_based=False,
)

In [20]:
# to replace the label with the right one
df = pd.read_csv('./data/anti-p40/GSE206285_data.csv')
df_meta = pd.read_csv('./data/anti-p40/GSE206285_metadata.csv')
df.loc[:,'GENE_SYMBOL'] = df.loc[:,'GENE_SYMBOL'].str.split('///')
df = df.explode('GENE_SYMBOL').reset_index(drop=True)
df.loc[:,'GENE_SYMBOL'] = df.loc[:,'GENE_SYMBOL'].apply(lambda x:x.strip())

# only select visit == baseline and treamt == ustekinumab, or visit == control group
df_sample = df_meta[((df_meta['visit'] == 'baseline') & (df_meta['treatment'] == 'ustekinumab')) | (df_meta['visit'] == 'control')]
gsm_list = df_sample.geo_accession.tolist() # target sample list
df_sample.loc[:,['visit','treatment','response']].value_counts(dropna=False)

visit     treatment    response
baseline  ustekinumab  No          315
                       Yes          49
control   NaN          NaN          18
dtype: int64

In [23]:
output_df.reset_index(inplace=True)
output_df.rename(columns={'index':'geo_accession'},inplace=True)
response = df_sample.loc[:,['geo_accession','response']]

output_df = output_df.merge(response,on='geo_accession',how='left')
output_df.drop('label',axis=1,inplace=True)
output_df.dropna(subset='response',inplace=True)
output_df.response = output_df.response.map({'No':0,'Yes':1})
output_df.rename({'response':'label'},axis=1,inplace=True)
output_df.to_csv('./output/scoring-5.csv',sep='\t')

In [ ]:
output_df, summary_df = do_radical_search(
    data = data,
    design = design,
    threshold=2.5,
    control="control",
    control_based=False,
)
output_df.to_csv('./output/scoring-2.5.csv',sep='\t')

In [ ]:
output_df, summary_df = do_radical_search(
    data = data,
    design = design,
    threshold=1,
    control="control",
    control_based=False,
)
output_df.to_csv('./output/scoring-1.csv',sep='\t')

In [ ]:
output_df, summary_df = do_radical_search(
    data = data,
    design = design,
    threshold=1.5,
    control="control",
    control_based=False,
)
output_df.to_csv('./output/scoring-1.5.csv',sep='\t')

In [ ]:
output_df, summary_df = do_radical_search(
    data = data,
    design = design,
    threshold=10,
    control="control",
    control_based=False,
)
output_df.to_csv('./output/scoring-10.csv',sep='\t')